In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


df=pd.read_csv("./rnaseq_FPKM_UQ_all.csv", low_memory=False,index_col=1)
df_tumor=df[df['sample_type']=='Tumor']

In [ ]:
# gene expression values # 1109 by 56716
df_tumor_gene=df_tumor.iloc[:,1:56717]
df_tumor_gene.index.names=['barcode']

# clinical information   # 1209 by 82
df_tumor_clinical=df_tumor.iloc[:,56717:]
df_tumor_clinical=df_tumor_clinical.set_index('barcode')


In [ ]:
# apply PCA to gene features

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# standardize gene features before applying PCA, make mean 0 and unit variance
sc=StandardScaler()
X_norm=sc.fit_transform(df_tumor_gene)

# apply PCA to gene features
pca=PCA()
X_pca=pca.fit_transform(X_norm)

temp=pca.explained_variance_ratio_.cumsum()

plt.plot(temp)
plt.xlabel('Number of Principle Components')
plt.ylabel('Proportion of Variance Explained')
plt.show();
 
# select the first 300 PCs and apply the transformation to data:

pca=PCA(n_components=300)
X_pca=pca.fit_transform(X_norm)

temp=pca.explained_variance_ratio_.cumsum()

data_x=X_pca

In [ ]:
# survival outcomes
data_y=df_tumor_clinical[["vital_status","days_to_last_follow_up","days_to_death"]]
time_to_event=[]
status=[]
for index, row in data_y.iterrows():
    if row['vital_status']=='dead':
        time_to_event.append(row['days_to_death'])
        status.append(1)
    elif row['vital_status']=='alive':
        time_to_event.append(row['days_to_last_follow_up'])
        status.append(0)
    else:
        time_to_event.append(None)
        status.append(None)

data_y=data_y.copy()
data_y['status']= pd.Series([x==1 for x in status], index=data_y.index)
data_y['time_to_event']= pd.Series(time_to_event, index=data_y.index)
data_y = data_y.drop(['days_to_last_follow_up','days_to_death','vital_status'], 1)

data_y = data_y.to_records(index=False)

In [ ]:
df=pd.DataFrame(data_y)
df['index_col'] = df.index
df_null = df[df.isnull().any(axis=1)]
df_null

In [ ]:
## Check columus with na outcomes

df=pd.DataFrame(data_y)
df['index_col'] = df.index
df_null = df[df.isnull().any(axis=1)]
df_null

data_x=np.delete(data_x,[537,944,1070],axis=0)
data_y=np.delete(data_y,[537,944,1070])

## Correct the follow uo days less than 0 to 0
for idx, item in enumerate(data_y['time_to_event']):
    if item < 0:
        data_y['time_to_event'][idx] = 0
data_y

df.groupby('status').count()


## Cox-PH model with Ridge Penalty

In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.model_selection import GridSearchCV

# tuning parameter alpha over grid search according to c-index

coxph = CoxPHSurvivalAnalysis()
grid_values={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}

grid_c = GridSearchCV(coxph, param_grid = grid_values, scoring = None)
grid_c.fit(data_x, data_y) 

print('Grid best parameter (max c-index): ', grid_c.best_params_)
print('Grid best score (c-index): ', grid_c.best_score_)

In [ ]:
# Apply Cox-PH model based on 5-fold 10-repeated CV using optimal alpha selected from grid search:

from sklearn.model_selection import RepeatedKFold

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=0) # 5-fold 10-repeated CV

c_index_train,c_index_test=[],[]

for train_index, test_index in rkf.split(data_x):
    x_train, x_test = data_x[train_index], data_x[test_index]
    y_train, y_test = data_y[train_index], data_y[test_index]
    coxph = CoxPHSurvivalAnalysis(alpha=float(grid_c.best_params_['alpha'])).fit(x_train, y_train)
    c_index_train.append(coxph.score(x_train,y_train))
    c_index_test.append(coxph.score(x_test,y_test))
    
print("Averaged c-index from 5-fold 10 repeated CV(training): {:.3f}".format(np.mean(c_index_train)))
print("Averaged c-index from 5-fold 10 repeated CV(test): {:.3f}".format(np.mean(c_index_test)))

## Elastic net Cox-PH model

In [ ]:
from sksurv.linear_model import CoxnetSurvivalAnalysis

alphas=np.arange(0.5,1,0.1)


estimator = CoxnetSurvivalAnalysis(n_alphas=5, alphas=alphas, l1_ratio=0.7)
estimator.fit(data_x, data_y)

estimator.score(data_x, data_y)


In [ ]:
## define a function for evaluating the performance of models during grid search using Harrell's concordance index
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y['status'], y['time_to_event'], prediction)
    return result[0]

param_grid = {'alpha': 2. ** np.arange(-12, 13, 2)}
cv = ShuffleSplit(n_splits=200, train_size=0.5, random_state=0)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=4, iid=False, refit=False,
                   cv=cv)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
gcv = gcv.fit(data_x, data_y)

print(gcv.best_score_)
print(gcv.best_params_)

## Finally, we retrieve all 200 test scores for each parameter setting and visualize their distribution by box plots.
def plot_performance(gcv):
    n_splits = gcv.cv.n_splits
    cv_scores = []
    for i, params in enumerate(gcv.cv_results_["params"]):
        validation_scores = np.empty(n_splits, dtype=float)
        for j in range(n_splits):
            validation_scores[j] = gcv.cv_results_["split%d_test_score" % j][i]
        name = "%.5f" % params["alpha"]
        cv_scores.append((name, validation_scores))

    sns.boxplot(pd.DataFrame.from_items(cv_scores))
    _, xtext = plt.xticks()
    for t in xtext:
        t.set_rotation("vertical")

plot_performance(gcv)

plt.savefig('./FTSVM_UQ_all_box.png')

In [ ]:
import seaborn as sns; sns.set()
ax = sns.heatmap(X_pca)
plt.show()

In [ ]:
# visualize the PCA-transformed data

from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import numpy

def plot_labelled_scatter(X, y, class_labels):
    num_labels = len(class_labels)

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    marker_array = ['o', '^', '*']
    color_array = ['#FFFF00', '#00AAFF', '#000000', '#FF00AA']
    cmap_bold = ListedColormap(color_array)
    bnorm = BoundaryNorm(numpy.arange(0, num_labels + 1, 1), ncolors=num_labels)
    plt.figure()

    plt.scatter(X[:, 0], X[:, 1], s=65, c=y, cmap=cmap_bold, norm = bnorm, alpha = 0.40, edgecolor='black', lw = 1)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    h = []
    for c in range(0, num_labels):
        h.append(mpatches.Patch(color=color_array[c], label=class_labels[c]))
    plt.legend(handles=h)

    

plot_labelled_scatter(X_pca, y, ['Normal','Tumor'])

plt.xlabel('First principle component')
plt.ylabel('Second principle component')
plt.title('TCGA BRCA data PCA(n_component = 2 )')
plt.show();

In [ ]:
# Visualizing PCA components
# Plotting the magnitude of each feature value for the first two principal component


fig = plt.figure(figsize=(8, 4))
plt.imshow(pca.components_, interpolation = 'none', cmap = 'plasma')
feature_names = list(df_gene.columns.values)

plt.gca().set_xticks(np.arange(-.5, len(feature_names)));
plt.gca().set_yticks(np.arange(0.5, 2));
plt.gca().set_xticklabels(feature_names, rotation=90, ha='left', fontsize=12);
plt.gca().set_yticklabels(['First PC', 'Second PC'], va='bottom', fontsize=12);

plt.colorbar(orientation='horizontal', ticks=[pca.components_.min(), 0, 
                                              pca.components_.max()], pad=0.65);

## Manifod Learning (for high-dim data visualization)

### Multidimensional scaling (MDS)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS


mds = MDS(n_components = 2)

X_mds = mds.fit_transform(X_norm)

plot_labelled_scatter(X_mds, y, ['Normal', 'Tumor'])

plt.xlabel('First MDS dimension')
plt.ylabel('Second MDS dimension')
plt.title('TCGA BRCA data MDS (n_components = 2)');

### t-SNE

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(random_state = 0,n_components = 2)

X_tsne = tsne.fit_transform(X_norm)

plot_labelled_scatter(X_tsne, y, 
    ['Normal', 'Tumor'])
plt.xlabel('First t-SNE feature')
plt.ylabel('Second t-SNE feature')
plt.title('TCGA BRCA data t-SNE');

In [ ]:
X_tsne.head()

## Clustering

### K-means

In [ ]:
from sklearn.cluster import KMeans 

kmeans=KMeans(n_clusters=5)
kmeans.fit(X_norm)

plot_labelled_scatter(X_norm, kmeans.labels_, ['Cluster1', 'Cluster2','Cluster3'])

In [ ]:
kmeans.labels_

### Agglomerative Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering


cls = AgglomerativeClustering(n_clusters = 3)
cls_assignment = cls.fit_predict(X_norm)

plot_labelled_scatter(X_norm, cls_assignment, 
        ['Cluster 1', 'Cluster 2', 'Cluster 3'])

In [ ]:
# Creating a dendrogram (using scipy), based on 10 selected samples

from scipy.cluster.hierarchy import ward, dendrogram
plt.figure()
dendrogram(ward(X_norm))
plt.show()

### DBSCAN

In [ ]:
# No need to specify number of clusters ahead
# Good for larger dataset, efficient
# Allow to identify noise points


from sklearn.cluster import DBSCAN


dbscan = DBSCAN(eps = 2, min_samples = 2)

cls = dbscan.fit_predict(X_norm)
print("Cluster membership values:\n{}".format(cls))

plot_labelled_scatter(X_norm, cls + 1, 
        ['Noise', 'Cluster 0', 'Cluster 1', 'Cluster 2'])
